Vaccination data is [here](https://covid.cdc.gov/covid-data-tracker/#vaccinations_vacc-total-admin-rate-total) and ICU data [here](https://coronavirus.jhu.edu/data/hospitalization-7-day-trend). The latter dataset is uploaded [here](https://github.com/RInterested/DATASETS/blob/gh-pages/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_State_Timeseries.csv). [This](https://stats.stackexchange.com/a/26779/67822) is what I intend to do.

In [1]:
install.packages('RCurl')
require(RCurl)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Loading required package: RCurl



Collecting the number of ICU and IP beds dedicated to COVID confirmed or suspected cases per state as a measure of serious disease:

In [2]:
url <- "https://raw.githubusercontent.com/RInterested/DATASETS/gh-pages/COVID19%20Reported%20Patient%20Impact%20and%20Hospital%20Capacity%20by%20State%20Timeseries"
dat <- read.csv(url)
# 13: ""inpatient_beds_used"  "
# 27: "staffed_icu_adult_patients_confirmed_and_suspected_covid"
# 31:"total_adult_patients_hospitalized_confirmed_and_suspected_covid" 
# 39 ""total_staffed_adult_icu_beds"   
data <- dat[,c(1,2,27,39,31,13)]
colnames(data) <- c('state','date','ICUpos','ICUb','IPpos','IPb')
data[,2] <- as.Date(data$date)
df <- data[!(rowSums(is.na(data))),]
df$ICUneg <- df[,4]-df[,3]
df$IPneg  <- df[,6]-df[,5]
df <- df[,c('state','date','ICUpos','ICUneg','ICUb','IPpos','IPneg','IPb')]
df <- df[order(df[,'state']), ]

# Adding IP's and ICU:
df$TOTpos <- df$ICUpos + df$IPpos
df$TOTneg <- df$ICUneg + df$IPneg
df$TOTb   <- df$ICUb + df$IPb
head(df)

,state,date,ICUpos,ICUneg,ICUb,IPpos,IPneg,IPb,TOTpos,TOTneg,TOTb
,<chr>,<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
73,AK,2020-07-29,11,32,43,51,1031,1082,62,1063,1125
381,AK,2020-08-29,9,37,46,36,826,862,45,863,908
393,AK,2020-12-20,20,106,126,106,856,962,126,962,1088
409,AK,2020-12-04,33,98,131,161,849,1010,194,947,1141
459,AK,2021-01-10,11,107,118,68,810,878,79,917,996
681,AK,2020-12-30,11,116,127,71,838,909,82,954,1036


Now the data in the last 30 days of delta wave in the US will be averaged state-wise. For example:

In [3]:
# Example Alabama:
n <- 30 # number of days to average
AL <- df[df$state=="AL",]
AL <- AL[order(as.Date(AL$date,format = "%d/%m/%Y")),]
row.names(AL) <- 1:nrow(AL)
round(colMeans(tail(AL[,c(3,4,5)],n)))
round(colMeans(tail(AL[,c(6,7,8)],n)))
round(colMeans(tail(AL[,c(9,10,11)],n)))

ICUpos ICUneg   ICUb 
   836    745   1581

IPpos IPneg   IPb 
 2873  9233 12106

TOTpos TOTneg   TOTb 
  3710   9978  13688

Performing the same averaging for all states:

In [4]:
states <- unique(df[,1])
m <- matrix(0,length(states),9)

for(i in 1:length(states)){
  temp <- df[df$state==states[i],]
  temp <- temp[order(as.Date(temp$date,format = "%d/%m/%Y")),]
  m[i,1:9] <- round(colMeans(tail(temp[,3:11],n)))
}

hosp <- cbind.data.frame(states,m)
hosp <- hosp[order(hosp[,'states']), ]
names(hosp) <- names(df)[c(1,3:11)]
head(hosp)

,state,ICUpos,ICUneg,ICUb,IPpos,IPneg,IPb,TOTpos,TOTneg,TOTb
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AK,35,89,123,152,931,1082,186,1019,1206
2,AL,836,745,1581,2873,9233,12106,3710,9978,13688
3,AR,457,547,1004,1307,5394,6701,1764,5941,7705
4,AS,0,7,7,0,75,75,0,82,82
5,AZ,478,1689,2167,1966,10277,12243,2444,11966,14410
6,CA,1935,5304,7240,8115,42998,51114,10051,48303,58353


Getting the vaccination data per state and tidying up the dataset:

In [6]:
y = url("https://raw.githubusercontent.com/RInterested/DATASETS/gh-pages/covid19_vaccinations_in_the_united_states.csv")
vaccines <- read.csv(y)
vac <- vaccines[,c(1,3)]
# 3: Doses Delivered per 100K
names(vac) <- c('state','jabsx100k')

s = url("https://raw.githubusercontent.com/RInterested/DATASETS/gh-pages/states.csv")
abbr <- read.csv(s)

vac <- vac[vac[,1] %in% abbr[,1],]

for(i in 1:nrow(vac)){
    vac[i,1] <- abbr[which(abbr[,1]==vac[i,1]),2]
}

vac[,2] <- as.numeric(vac[,2])

d <- merge(hosp,vac, by="state")
head(d)

,state,ICUpos,ICUneg,ICUb,IPpos,IPneg,IPb,TOTpos,TOTneg,TOTb,jabsx100k
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AK,35,89,123,152,931,1082,186,1019,1206,135083
2,AR,457,547,1004,1307,5394,6701,1764,5941,7705,127641
3,AS,0,7,7,0,75,75,0,82,82,111404
4,AZ,478,1689,2167,1966,10277,12243,2444,11966,14410,131204
5,CA,1935,5304,7240,8115,42998,51114,10051,48303,58353,142990
6,CO,273,1079,1353,777,7224,8002,1051,8304,9354,137594


Checking whether the logistic regression of covid cases in the ICU and in the hospital (total ICU and IP) regressed over the vaccination status of the population in different states is significant:

In [7]:
fit <- glm(cbind(d$ICUpos,d$ICUneg) ~ scale(d$jabsx100k), 
           family='binomial')

summary(fit)


Call:
glm(formula = cbind(d$ICUpos, d$ICUneg) ~ scale(d$jabsx100k), 
    family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-27.566   -7.525   -2.758    2.623   39.715  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -0.871522   0.007831 -111.30   <2e-16 ***
scale(d$jabsx100k) -0.193593   0.010832  -17.87   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 8346.2  on 51  degrees of freedom
Residual deviance: 8024.5  on 50  degrees of freedom
AIC: 8375.2

Number of Fisher Scoring iterations: 4


The scaling and centering is meant to estimate the effect size. From [here](https://www.theanalysisfactor.com/effect-size-statistics-logistic-regression/):

*For example, let’s say you’re doing a logistic regression for a ecology study on whether or not a wetland in a certain area has been infected with a specific invasive plant.  Predictors include water temperature in degrees Celsius, altitude, and whether the wetland is a fen or a marsh.*

*If the odds ratio for water temperature is 1.12, that means that for each one-degree Celsius increase in water temperature, the odds of the wetland having the invasive plant species is 1.12 times as big, after controlling for the other predictors.*

*That odds ratio is an unstandardized effect size statistic.  It tells you the direction and the strength of the relationship between water temperature and the odds that the plant is present.*

*It’s unstandardized because it’s based on the units of temperature.**

I realize no ecologist would do so, but if the water were measured in degrees Fahrenheit, that odds ratio would have a different value.  The direction and the strength of the relationship would be the same, but the statistic would be evaluated on a different scale* 

In [8]:
exp(cbind(OR = coef(fit), confint(fit)))

Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.4183142,0.4119365,0.4247772
scale(d$jabsx100k),0.8239930,0.8066747,0.8416660


Intercept: The odds of being a covid positive patient in the ICU if zero vaccinations is 40%.

Slope: A $1$ SD increase in the vaccination status of the state, decreases 20% the odds of being in the ICU.

Now for inpatients and ICU combined (i.e. hospitalizations):

In [11]:
fit2 <- glm(cbind(d$TOTpos,d$TOTneg) ~ scale(d$jabsx100k), family='binomial')
summary(fit2)
exp(cbind(OR = coef(fit2), confint(fit2)))


Call:
glm(formula = cbind(d$TOTpos, d$TOTneg) ~ scale(d$jabsx100k), 
    family = "binomial")

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-50.845  -16.359   -8.674    2.583   94.627  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)        -1.489019   0.003298 -451.45   <2e-16 ***
scale(d$jabsx100k) -0.153360   0.004552  -33.69   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 35084  on 51  degrees of freedom
Residual deviance: 33944  on 50  degrees of freedom
AIC: 34382

Number of Fisher Scoring iterations: 4


Waiting for profiling to be done...



,OR,2.5 %,97.5 %
(Intercept),0.2255940,0.2241393,0.2270560
scale(d$jabsx100k),0.8578211,0.8502000,0.8655079


The odds of being a covid positive patient admitted to a hospital is 80% without any vaccines.

The odds of being a covid positive patient admitted to the hospital is decreased $15\%$ as the vaccination of the state increases by $1$ SD.